In [3]:
import pandas as pd
import glob
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
path = "d:\\Summer Practicum\\1_Data\\On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2023_All_Months"
all_files = glob.glob(os.path.join(path,'*.csv'))

# Read all files in the Data folder
df = pd.concat((pd.read_csv(f,low_memory = False) for f in all_files), ignore_index=True)

In [5]:
df['DestAirportID'].value_counts()

DestAirportID
10397    322013
13930    295365
11292    283281
11298    277590
11057    213046
          ...  
13983        91
10643        47
12320        22
13829         4
10590         1
Name: count, Length: 374, dtype: int64

In [7]:
df_sorted = df.sort_values(by = ['DestAirportID', 'FlightDate', 'ArrTime'])
df_sorted

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
1980730,2022,2,4,1,5,2022-04-01,AA,AA_CODESHARE,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1890913,2022,2,4,1,5,2022-04-01,DL,DL_CODESHARE,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2116641,2022,2,4,1,5,2022-04-01,G4,G4,20368,G4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2119741,2022,2,4,1,5,2022-04-01,G4,G4,20368,G4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1742936,2022,2,4,1,5,2022-04-01,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6825910,2023,1,3,30,4,2023-03-30,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
6826662,2023,1,3,31,5,2023-03-31,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
6806779,2023,1,3,31,5,2023-03-31,DL,DL_CODESHARE,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
6826798,2023,1,3,31,5,2023-03-31,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [10]:
df_grouped2 = df.groupby(['DestAirportID', 'FlightDate']).agg({'ArrDelay': 'mean'})


In [13]:
df_grouped2.sort_values(by = ['FlightDate', 'ArrDelay'], ascending = False).reset_index()

,DestAirportID,FlightDate,ArrDelay
0,14150,2023-03-31,141.0
1,13377,2023-03-31,139.6
2,12119,2023-03-31,131.0
3,14025,2023-03-31,97.5
4,12012,2023-03-31,82.5
...,...,...,...
129099,13211,2022-04-01,-27.5
129100,13241,2022-04-01,-28.5
129101,15569,2022-04-01,-40.5
129102,10245,2022-04-01,NaN
